![Screenshot 2023-02-22 164309.png](Screenshot_2023-02-22_164309.png)

I 1978 lurte 3 smarte menn fra MIT på hvordan en fiktiv bank kunne bruke en server (time-shared service) til å kalkulere sensitive data fra sine låntakere uten å gi fra seg noe informasjon. Et av forslagene var å bruke homomorfisk kryptering.

Dette var kun teoretisk i 30 år før Gentry laget det først FHE rammeverket i 2009. I 2011 kom Microsoft ut med et paper “Can Homomorphic Encryption be Practical?”. I 2018 kom Microsoft ut med en implementasjon kalt Simple Encrypted Arithmetic Library (SEAL). Og i 2022 kom Edge med Password Monitor som bruker FHE til å sjekke om passordet ditt finnes i lekkede passord-databaser. (Chrome kom kort tid etterpå med en hash-basert løsning).


## RSA
1. Bob selects two large prime $p<<0$ and $q<<0$ and calclate $n=p \times q$
2. Bob calculate $\phi(n) = (p-1)(p+2)$. Choose $e$ such that $\gcd(e, \phi(n))=1$.
3. Bob calculate private key $d$ to be the modular inverse of $e$, in other words $de \equiv 1 (\mod \phi(n))$.
4. Bob sends the public key $p_k = n,p$
5. Anna encrypts message $m$ to ciphertext $c \equiv m^e (\mod n)$. And sends it.
6. Bob computes the message $c^d \equiv m (\mod n)$

In [12]:
from math import gcd
# Bobs computer
p, q, e = 11, 17, 3 # pretend that p and 1 are large (4096 bits)
n = p * q #187
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent = 42
cipher = message_sent**e % n
print(f'{cipher=}')

# Bobs computer
message_recieved = cipher**d % n
print(f'{message_recieved=}, {message_recieved==message_sent=}')


n=187
1
cipher=36
message_recieved=42, message_recieved==message_sent=True



# Homomorphism in RSA

In [17]:
from math import gcd
# Bobs computer
p, q, e = 11, 17, 3
#p, q = 101, 103
#e = 7
n = p * q #187
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)=}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent_j = 10
message_sent_k = 42
cipher_j = message_sent_j**e % n
cipher_k = message_sent_k**e % n
print(f'{cipher_j=}, {cipher_k}')

#Homomorphic calculation
cipher_product = cipher_k * cipher_j

# Bobs computer
message_recieved = cipher_product**d % n
print(f'{message_recieved=}, {message_recieved==message_sent=}')

n=187
gcd(e, phi_n)=1
cipher_j=65, 36
message_recieved=46, message_recieved==message_sent=False


# Takebacks
Ikke evig multiplikasjoner, `ciper_product`$ \equiv  m_j \times m_j \bmod {n}$

# Paillier
Generate keys
$$
\text{primes} \; p,q, \; \text{random integers} \;g \\
\begin{align}
\Z_n &= {0,1,2, \ldots, n-1} \\
p,q &\in \N_{p(rime)} | \gcd ( pq, (p−1)(q−1))=1 \\
n &=pq\\
\lambda &= \text{lcm} (p-1,q-1) \\
g   &\in_R \Z_{n^2}^* \\
L(x)& = {{x-1}\over n} \\
\mu &=(L(g^{\lambda }{\bmod \; n}^{2}))^{{-1}}{\bmod \; n}
b \\
\text{public key} =  k_{pub} &= (n, g)  \\
\text{private key} = k_{priv} &= (\lambda, \mu)  \\
\end{align}
$$

Do some encryption magic.
$$
\begin{align}
\text{Encrypt one message}\nonumber \\
\text{message} &= m \in\Z_n \\
\text{random seed} &= r \in_R \N | \gcd(r,n)=1 \\
\text{ciphertext} &=c =\mathcal{E}(m) = g^m r^n &\bmod\; n^2 \\
\nonumber \\
\text{Encrypt two messages} \nonumber \\
\mathcal{E}(m_1) &= g^{m_1} r_1^n&\bmod \;n^2 \\
\mathcal{E}(m_2) &= g^{m_2} r_2^n&\bmod \;n^2 \\
\nonumber \\
\text{Homomorphic mulitplication of two ciphertexts} \nonumber \\
\mathcal{E}(m_1) \times \mathcal{E}(m_2) &= (g^{m_1} r_1^n)(g^{m_2} r_2^n)&\bmod \;n^2 \\
&= g^{m_1 + m_2} (r_1r_2)^n &\bmod\; n^2 \\
&= \mathcal{E}(m_1 + m_2)\\
\nonumber \\
\text{Homomorphic mulitplication of one ciphertext and non-encrypted constant}= k \nonumber \\
\mathcal{E}(m) \times g^k = g^m r^n \times g^k\\
                         &= g^{m+k} r^n\\
\nonumber \\
\text{Decrypt} \nonumber \\
m &=L(c^{\lambda} \bmod \; n`2) \times \mu \bmod \; n
\end{align}
$$

In [21]:
from phe import paillier
public_key, private_key = paillier.generate_paillier_keypair()
messages = [33, 66]
cipher_texts = [public_key.encrypt(x) for x in messages]
cipher_texts.append(cipher_texts[0]+cipher_texts[1])
cipher_texts.append(cipher_texts[0]*10)
decrypted = [private_key.decrypt(x) for x in cipher_texts]
print(f'{decrypted=}')

decrypted=[33, 66, 99, 330]


Notater:


# Hva kan dette brukes til?
* Anonym avstemning
* Anonym treningsdata til K.I.
* Kan kanskje *ikke* brukes der du *ikke* vil ha *mallebility*.
    * Noen ganget din krypterte IOU liste med 2.
* ?? Kanskje ikke så mye mer...?
* Men hva om vi fikk til både $\times$ OG $+$ ???

# $\{ \times, + \} \rightarrow \{\wedge, \vee, \oplus, \; \neg \} \Leftrightarrow \{\&,|,!,||\} $

$$
\begin{align}
a,b &\in\{0,1\}\\
a\oplus b & \Leftrightarrow a+b &\bmod 2 \\
a\wedge b & \Leftrightarrow a\times b \\
a \vee b & \Leftrightarrow a + b + (a \times b) &\bmod 2
\end{align}
$$

|||$ a\oplus b$|$a \wedge b$|$a \vee b $|
|--|--|--|--|--|
|$a$|$b$|$a+b\bmod 2$|$a\times b$| $a + b + (a \times b) \bmod 2$|
| 0 | 0 |$0+0=0$     |$0 \times 0 = 0$|$0+0+(0 \times0) = 0$|
| 0 | 1 |$0+1=1$     |$0 \times 1 = 0$|$0+1+(0\times 1) =1$|
| 1 | 0 |$1+0=1$     |$1 \times 0 = 0$|$1+0+(1\times 0) =1$|
| 1 | 1 |$1+1= 2 \equiv0\bmod2$|$1 \times 1 = 1$|$1+1+(1\times 1)=3\equiv1 \bmod 2$|

# FHE **F**ully **H**omomorphic **E**ncryption
* Si noe om Noice budget
* Når du har nådd syøt-budsjettet ditt, kan du ta imot tallet ditt, decryptere det, og sende det inn igjen krypter.
* Bootstrapping: Å dekryptere kryptert data kryptert.
* Noe om first generation, second og third..
* 4th generation FHE:  Cheon, Kim, Kim and Song (CKKS)
    * Homomorphic Encryption for Arithmetic of Approximate Numbers 2016, ASIACRYPT 2017
    * Floats.
    * In short, our encoding function is given by:
    * ![Screenshot](CKKS_enc.png)
    * > ...we show that our scheme can be applied to the efficient evaluation of transcendental functions such as multiplicative inverse, exponential function, logistic function and discrete Fourier transform.
    * > The primary open problem is finding way to convert our scheme to a fully homomorphic scheme using bootstrapping.

    

# LPSE **L**ightweight **p**assword-**s**trength estimation for password meters

Bruker Cosine Similarity for å se på forskjellen mellom 2 vektorer. Der vektor 1 er et simulert sterkt passord og vektor 2 er passordet som skal måles

$$
\begin{align}
\text{Vectors}\;A,B
\text{cosine similarity} = S_C(A,B) :&= cos(\phi)\\
 &= {{A \cdot B}\over {||A|| \times ||B||}} \\
 &={ {\sum_{n=1}^n A_i \cdot B_i}\over{\sqrt { \sum_{n=1}^n A_i^2} \cdot \sqrt{ \sum_{n=1}^n A_i^2}}}
\end{align}
$$


Notater:
ref: https://eprint.iacr.org/2016/421.pdf
